In [ ]:
import sys
sys.path.insert(0, '../../')

In [ ]:
import bd_calc
import graph
import importlib
import math
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import random
import topo
import typing
import utils
import yaml

matplotlib.rc('font', family='Times New Roman', size=18)

In [ ]:
import all_cloud_cut
import best_cut
import flow_cut
import greedy_cut

def unit_size_cb(r: int):
    return 10000 * math.pow(10, random.randint(0, 1))

In [ ]:
topo_file = "../../samples/1e3h.yaml"
source_selector_dict = {'rasp1': 8, 'rasp2': 8, 'rasp3': 8}

sc = topo.Scenario.from_dict(
    yaml.load(open(topo_file, "r").read(), Loader=yaml.Loader)
)
while True:
    source_selector = graph.SourceSelector(source_selector_dict)
    gen_args ={
            "total_rank": random.randint(3, 7),
            "max_node_per_rank": random.randint(1, 2),
            "max_predecessors": random.randint(1, 2),
            "mi_cb": lambda: 1,
            "memory_cb": lambda: int(2e8),
            "unit_size_cb": unit_size_cb,
            "unit_rate_cb": lambda: random.randint(10, 20),
            "source_hosts": source_selector,
            "sink_hosts": ["cloud1"],
        }
    g = graph.GraphGenerator("g-debug", **gen_args).gen_dag_graph()
    greedy2_result = greedy_cut.greedy_cut2(sc, [g])[0]
    greedy2_bd = bd_calc.bd_calc(g, greedy2_result[0], greedy2_result[1])
    all_cloud_result = all_cloud_cut.all_cloud_cut(sc, [g])[0]
    all_cloud_bd = bd_calc.bd_calc(g, all_cloud_result[0], all_cloud_result[1])
    if greedy2_bd > all_cloud_bd:
        for u, v, d in g.get_edges():
            print('{} -- {} --> {}'.format(u, int(d['unit_size']*d['per_second']/1e3), v))
        print(greedy2_bd / 1e3)
        print(all_cloud_bd / 1e3)
        print(greedy2_result)
        print(all_cloud_result)
        break